In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('TweetData.csv',encoding='latin-1',header=None)

In [3]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df.columns =['target', 'id', 'date', 'flag', 'user', 'text']

In [5]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
df = df.sample(frac=0.01,random_state=1)

In [7]:
df.count()

target    16000
id        16000
date      16000
flag      16000
user      16000
text      16000
dtype: int64

In [8]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [9]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Vikrant
[nltk_data]     Yadav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Vikrant
[nltk_data]     Yadav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\Vikrant
[nltk_data]     Yadav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+", "", text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize text
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

# Apply the cleaning function to all tweets
df['cleaned_text'] = df['text'].apply(clean_text)

# Drop unnecessary columns
df = df[['target', 'cleaned_text']]

In [24]:
df.head()

,target,cleaned_text
514293,0,miss nikki nu nu already shes always needed th...
142282,0,dream last night remember sign clearly told ge...
403727,0,girlyghost ohh poor sickly hugs hope feel litt...
649503,0,raining
610789,0,misskeribaby wish la right


In [28]:
# Replace 4 with 1 for positive sentiment, and keep 0 for negative sentiment
df['target'] = df['target'].replace({4: 1, 0: 0})

In [32]:
df.tail()

,target,cleaned_text
133475,0,really really dont want leave go work
638631,0,measly updates
740314,0,great time catching close mates nite dshibis p...
367475,0,pouring outside whole rain thing depressing fb
1116970,1,songs ipod love


In [34]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and labels (y)
x = df['cleaned_text']
y = df['target']

# Split the data into training and testing sets (80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(f"Training set size: {len(x_train)}")
print(f"Testing set size: {len(x_test)}")

Training set size: 12800
Testing set size: 3200


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer (limit to 5000 most frequent words)
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data, and transform the test data
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

In [38]:
from sklearn.svm import SVC

# Initialize the SVM model with RBF kernel
svm_model = SVC(kernel='rbf', C=1.0, gamma='auto')

# Train the model on the TF-IDF transformed data
svm_model.fit(x_train_tfidf, y_train)

SVC(gamma='auto')

In [40]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for tuning SVM
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'rbf']
}

# Initialize GridSearchCV for SVM
grid_search = GridSearchCV(SVC(), param_grid, cv=5, verbose=2)

# Fit grid search
grid_search.fit(x_train_tfidf, y_train)

# Get the best parameters
print(f"Best Parameters: {grid_search.best_params_}")

# Use the best estimator from grid search
svm_model = grid_search.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   8.2s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   8.4s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   8.5s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   8.3s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   8.1s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   9.6s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   9.5s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   9.5s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   9.2s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   9.3s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=   8.2s
[CV] END ...................C=0.1, gamma=auto, k

In [44]:
from sklearn.metrics import accuracy_score, classification_report

# Predict the sentiment for the test set
y_pred_svm = svm_model.predict(x_test_tfidf)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Model Accuracy: {accuracy_svm}")

# Display a detailed classification report
print(classification_report(y_test, y_pred_svm))

SVM Model Accuracy: 0.7390625
              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1571
           1       0.74      0.75      0.75      1629

    accuracy                           0.74      3200
   macro avg       0.74      0.74      0.74      3200
weighted avg       0.74      0.74      0.74      3200



In [46]:
import joblib

# Save the trained SVM model
joblib.dump(svm_model, 'svm_sentiment_model.pkl')

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [52]:
# Calculate the number of positive predictions
positive_predictions_svm = (y_pred_svm == 1).sum()
total_predictions_svm = len(y_pred_svm)

# Calculate the support index as the percentage of positive sentiment
support_index_svm = (positive_predictions_svm / total_predictions_svm) * 100

print(f"Estimated Support Index: {support_index_svm}%")

Estimated Support Index: 52.0%


In [54]:
# Load the model and vectorizer
svm_model = joblib.load('svm_sentiment_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Example of predicting sentiment on a new tweet
new_tweet = ["I love the campaign promises of this candidate!"]
new_tweet_tfidf = vectorizer.transform(new_tweet)
prediction = svm_model.predict(new_tweet_tfidf)

print(f"Predicted Sentiment: {prediction}")

Predicted Sentiment: [1]
